<a href="https://colab.research.google.com/github/jaewon078/Stock-Trader-Chatbot/blob/main/Stock_Trader_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Stock Trader Chatbot by Jaewon Lee</h1>


<h2> General Information: </h2>

* Model: GPT 4o
* Colab Environment: CPU


## Setting up libraries and the model

In [1]:
# Install the necessary libraries
!pip install -q openai==1.44.1
!pip install -q gradio==4.43.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/

In [2]:
# Import libraries
import openai
import random
import gradio

In [3]:
# Insert OpenAI API Key here (you will be prompted)
# API keys are available at https://platform.openai.com/account/api-keys
from getpass import getpass
openai.api_key=getpass("OpenAI API Key: ")

# We are using GPT-4o as our model
model = "gpt-4o"

OpenAI API Key: ··········


In [4]:
# Setting up the context so the model acts like an experienced stock trader

context = [
    {
        'role': 'system',
        'content': """
You are an experienced stock trader with over 20 years of experience in the financial markets. Your expertise spans various sectors including technology, healthcare, energy, and consumer goods. You have a deep understanding of fundamental and technical analysis, market trends, and economic indicators.

Your role is to provide advice and insights on stock trading to users. This includes:
1. Analyzing specific stocks or market sectors when requested
2. Explaining trading strategies and concepts
3. Discussing current market trends and their potential impacts
4. Offering general advice on portfolio management and risk mitigation

When giving advice, always:
1. Emphasize that your advice is for informational purposes only and should not be considered as financial advice
2. Encourage users to do their own research and consult with a licensed financial advisor before making investment decisions
3. Explain the reasoning behind your analysis or recommendations
4. Discuss both potential upsides and risks associated with any strategy or stock
5. Use clear, concise language and explain any technical terms you use

Remember to stay up-to-date with the latest market news and trends. If asked about specific recent events, you can provide general analysis based on typical market behavior, but remind the user that you don't have real-time data and they should verify current information.

Never provide guarantees about stock performance or make promises about returns. Always stress the importance of diversification and risk management in investing.

Respond directly to user queries without unnecessary affirmations or filler phrases. Be concise but thorough, offering to elaborate if further information may be helpful.
"""
    },
    {
        'role': 'assistant',
        'content': "Hello! I'm your experienced stock trading assistant. How can I help you with your investment queries today?"
    }
]

## Setting up the logic for maintaining the memory of the conversation

In [5]:
# Generates the next AI response in a conversation using OpenAI's Chat API
#
# Parameters:
# messages: List of message dictionaries representing the conversation history
#           Each dictionary should have 'role' (system/user/assistant) and 'content' keys
# temperature: Float between 0 and 2. Lower values make output more focused and deterministic
#
# Returns:
# str: The content of the AI's response message

def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

## Setting up Gradio

In [6]:
# Processes a new user message in a Gradio chat interface and generates an AI response
#
# This function is designed to work with Gradio's chat interface. It takes the current
# user message and the conversation history, formats them into the expected structure
# for the OpenAI API, and then generates the next AI response.
#
# Parameters:
# message: str - The current user message to be processed
# history: list - A list of tuples, each containing (user_message, assistant_message)
#                 representing the chat history
#
# Returns:
# str - The AI's response to the current message

def gradio_chat(message, history):
    # Initialize the chat history with the predefined context
    history_chat = context

    # Add each user and AI message to the chat history
    for user, assistant in history:
        history_chat.append({"role":"user", "content":user})
        history_chat.append({"role":"assistant", "content":assistant})

    # Add the current user message to the chat history
    history_chat.append({"role":"user", "content":message})

    # Generate and return the AI's response using the complete chat history
    return continue_conversation(history_chat, 0)

In [7]:
# Customized gradio textbox
InputText = gradio.Textbox(label="Send", info="Your response here.", scale= 6)

## Final Result

In [8]:
gradio.ChatInterface(gradio_chat,
                 textbox=InputText,
                 retry_btn=None,
                 undo_btn=None,
                 title="Stock Trader Chatbot",
                submit_btn="Send").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1bd44908fc41fb79b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
